# MIMIC data

## Get MIMIC_compile, which consists of both vital and inv data 

In [2]:
%cd /content/drive/My Drive/ColabNotebooks/MIMIC/Extract/MEEP/Extracted_Feb_2023

/content/drive/My Drive/ColabNotebooks/MIMIC/Extract/MEEP/Extracted_Feb_2023


In [3]:
# MIMIC data compile Feb 2022 with time window 
import pandas as pd
vital_train = pd.read_hdf('./MIMIC_split_2.hdf5', key='vital_train')
vital_dev = pd.read_hdf('./MIMIC_split_2.hdf5', key='vital_dev')
vital_test = pd.read_hdf('./MIMIC_split_2.hdf5', key='vital_test')

inv_train = pd.read_hdf('./MIMIC_split_2.hdf5', key='inv_train')
inv_dev = pd.read_hdf('./MIMIC_split_2.hdf5', key='inv_dev')
inv_test = pd.read_hdf('./MIMIC_split_2.hdf5', key='inv_test')

static_train = pd.read_hdf('./MIMIC_split_2.hdf5', key='static_train')
static_dev = pd.read_hdf('./MIMIC_split_2.hdf5', key='static_dev')
static_test = pd.read_hdf('./MIMIC_split_2.hdf5', key='static_test')

<ipython-input-3-e7e49223ea28>:3: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  vital_train = pd.read_hdf('./MIMIC_split_2.hdf5', key='vital_train')
<ipython-input-3-e7e49223ea28>:3: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  vital_train = pd.read_hdf('./MIMIC_split_2.hdf5', key='vital_train')
<ipython-input-3-e7e49223ea28>:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  vital_dev = pd.read_hdf('./MIMIC_split_2.hdf5', key='vital_dev')
<ipython-input-3-e7e49223ea28>:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  vital_dev = pd.read_hdf('./MIMIC_split_2.hdf5', key='

In [4]:
ID_COLS = ['subject_id', 'hadm_id', 'stay_id']
vital_train = vital_train.join(inv_train, on=ID_COLS + ['hours_in'])
vital_dev = vital_dev.join(inv_dev, on=ID_COLS + ['hours_in'])
vital_test = vital_test.join(inv_test, on=ID_COLS + ['hours_in'])
# sort values 
def sort_df(df, sortbylevel=2):
    df.sort_index(axis=0, level=sortbylevel, inplace=True)
df_list = [vital_train , vital_dev, vital_test, static_train, static_dev, static_test]
for df in df_list:
    sort_df(df)

# list of arrays here for vitals and sofa 
idx = pd.IndexSlice
def get_ind_array(df):
    df_array = [df.loc[idx[:,:,m,:], :].values for i, m in enumerate(sorted(set(df.index.get_level_values(2))))]
    return df_array
[vtrain, vdev, vtest] = [get_ind_array(df) for df in [vital_train, vital_dev, vital_test]]

## take out tail and head for model prediction 
def take_vital_head(df):
    vital_list = [df[i].transpose(1, 0).astype(float) for i in range(len(df))]
    return vital_list

# def take_sofa_tail(df):
#     sofa_list = [df[i][24:, :] for i in range(len(df))]
#     return sofa_list

# def take_sofa_head(df):
#     sofa_list = [df[i][:-24, :] for i in range(len(df))]
#     return sofa_list

def take_complete(df):
    complete_list = [df.loc[idx[:,:,m,:], ['hospital_expire_flag', 'mort_icu', 'max_hours']].values.squeeze() for i, m in enumerate(sorted(set(df.index.get_level_values(2))))]
    return complete_list 

[vhead_train, vhead_dev, vhead_test] =  [take_vital_head(df) for df in [vtrain, vdev, vtest]]
[st_train, st_dev, st_test] =  [take_complete(df) for df in [static_train, static_dev, static_test]]

In [ ]:
import numpy as np 
MIMIC_compile_0218_2023_2= {'train_head': vhead_train, 'static_train_filter': st_train, \
                        #  'train_sofa_tail': stail_train, 'train_sofa_head': shead_train,\
                         'dev_head': vhead_dev, 'static_dev_filter': st_dev, \
                        #  'dev_sofa_tail': stail_dev, 'dev_sofa_head': shead_dev, \
                         'test_head': vhead_test, 'static_test_filter': st_test, \
                        #  'test_sofa_tail': stail_test, 'test_sofa_head': shead_test, \
                         }
np.save('/content/drive/MyDrive/ColabNotebooks/MIMIC/Extract/MEEP/Extracted_Feb_2023/MIMIC_compile_0218_2023_2.npy', MIMIC_compile_0218_2023_2, allow_pickle=True) 

## Keep relavant static data for fusion and target (SOFA score) for prediction

In [ ]:
# process static data
scol_keep = ['gender', 'age', 'ethnicity', 'hospital_expire_flag', 'max_hours', 'myocardial_infarct', 'congestive_heart_failure', 'peripheral_vascular_disease',
 'cerebrovascular_disease', 'dementia', 'chronic_pulmonary_disease', 'rheumatic_disease', 'peptic_ulcer_disease', 'mild_liver_disease', 'diabetes_without_cc', 'diabetes_with_cc',
 'paraplegia', 'renal_disease', 'malignant_cancer', 'severe_liver_disease', 'metastatic_solid_tumor', 'aids']
def categorize_ethnicity(ethnicity):
    if 'AMERICAN INDIAN' in ethnicity:
        ethnicity = 'AMERICAN INDIAN'
    elif 'ASIAN' in ethnicity:
        ethnicity = 'ASIAN'
    elif 'WHITE' in ethnicity:
        ethnicity = 'WHITE'
    elif 'HISPANIC' in ethnicity:
        ethnicity = 'HISPANIC/LATINO'
    elif 'BLACK' in ethnicity:
        ethnicity = 'BLACK'
    else: 
        ethnicity = 'OTHER'
    return ethnicity

static.loc[:, 'ethnicity'] = static['ethnicity'].apply(categorize_ethnicity)
g_map = {'F': 0, 'M': 1}
static.loc[:, 'gender'] = static.loc[:, 'gender'].map(g_map)
ind_1 = static.columns.get_loc('age')
age_mean = static.iloc[:,ind_1].mean()
age_stds = static.iloc[:,ind_1].std()
static.iloc[:,ind_1] =  (static.iloc[:,ind_1] - age_mean)/ (age_stds)
static_cat = pd.get_dummies(static, columns = ['ethnicity'])
static = static.loc[: , scol_keep]


## Get MIMIC_target, which contains SOFA score data for prediction